In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns

from IPython.display import display
from ipywidgets import interact, interactive,interact_manual, fixed, HBox, Label, VBox, GridspecLayout, Layout
from scipy.stats import poisson


plt.style.use('seaborn-whitegrid')
pd.set_option('display.max_columns', 50)
%matplotlib inline

In [ ]:
# Load CSVs
path = './data/'
provider_profiles = pd.read_csv(path+'pp_contacts decisions.csv', index_col='Provider Profiles')
arrivals = pd.read_csv(path+'dp_arrivals.csv')
other = pd.read_csv(path+'dp_other.csv')
rooms = pd.read_csv(path+'dp_rooms.csv')
wRVU = pd.read_csv(path+'dp_wRVU.csv')

arrivals['Date'].fillna(value='Week Average', axis=0, inplace=True)
rooms['Date'].fillna(value='Week Average', axis=0, inplace=True)

# Arrival Distributions

In [ ]:
A_averages = arrivals[['12:00:00 AM',
       '01:00:00 AM', '02:00:00 AM', '03:00:00 AM', '04:00:00 AM',
       '05:00:00 AM', '06:00:00 AM', '07:00:00 AM', '08:00:00 AM',
       '09:00:00 AM', '10:00:00 AM', '11:00:00 AM', '12:00:00 PM',
       '01:00:00 PM', '02:00:00 PM', '03:00:00 PM', '04:00:00 PM',
       '05:00:00 PM', '06:00:00 PM', '07:00:00 PM', '08:00:00 PM',
       '09:00:00 PM', '10:00:00 PM', '11:00:00 PM']].iloc[0].tolist()

In [ ]:
arrivalsT = arrivals.copy()
arrivalsT.drop(columns=['Year', 'Event'], axis=0, inplace=True)
new_cols = arrivalsT['Department Profile'].values + '_' +  arrivalsT['Date'].values
arrivalsT = arrivalsT.transpose()
arrivalsT.columns = new_cols
arrivalsT.drop(['Department Profile','Date'], axis=0, inplace=True)
arrivalsT.head(3)

### Arrivals per hour by Day

In [ ]:
ax = arrivalsT.plot(kind='line', y=arrivalsT.columns, use_index=True, xticks=[x for x in range(0,24)], legend=False,figsize=(20,10))
ax.set_xticklabels(arrivalsT.index.tolist(), rotation=50);

### Interactive Arrivals per Hour

In [ ]:
@interact
def scatter_plot(y=list(arrivalsT.columns)):
    ax = arrivalsT.plot(kind='line', y=y, use_index=True, xticks=[x for x in range(0,24)], figsize=(20,10));
    ax.set_xticklabels(arrivalsT.index.tolist(), rotation=50)

### Arrival Probability: Poisson Distribution

The Poisson Process is a model for infrequent events — where we know the average time between events but the actual time between any two events is randomly distributed (stochastic). Each event is independent of all other events, which means we can’t use the time since the last event to predict when the next event will occur. A Poisson model is an extension of a binomial model to situations where the number of expected successes is much less than the number of trials.

In [ ]:
arrivals[arrivals['Department Profile']=='A']

In [ ]:
@interact
def poission(times=arrivalsT['A_Week Average']):
    mu = times
    mean, var, skew, kurt = poisson.stats(mu, moments='mvsk')
    low = poisson.ppf(0.01, mu)
    high = poisson.ppf(0.99, mu)
    x = np.arange(low,high)
    y = poisson.pmf(x, mu)
    fig, ax = plt.subplots(1, 1, figsize=(20,10))
    ax.plot(x, y , 'bo', ms=14, label='poisson pmf');
    ax.vlines(x, 0, y, colors='b', lw=5, alpha=0.3);

### Average Hourly Provider Contacts and Decisions

In [ ]:
aver_contacts = []
aver_decs = []
for i in provider_profiles.columns:
    if i.startswith('Con'):
        aver_contacts.append(provider_profiles[i].mean())
    if i.startswith('Dec'):
        aver_decs.append(provider_profiles[i].mean())
        
contacts = pd.DataFrame(columns=['Contact', 'Decisions'])
contacts['Contact'] = aver_contacts
contacts['Decisions'] = aver_decs

In [ ]:
plt.figure(figsize=(16, 6))
plot = sns.lineplot(data=contacts, palette="tab10", linewidth=3.5)
plot.set(xlabel='Hour', ylabel='Average')
plt.title("Average Hourly Provider Contacts and Decisions");

### Optimizer

In [ ]:

N_samples = 25
x=np.linspace(-2,2,N_samples)

def f(x,a,mu,sigma):
    r=a*np.exp(-(x-mu)**2/(2*sigma**2))
    return (r)
def func(amplitude,ideal_mu,ideal_sigma,noise_sd,noise_mean):
    r=amplitude*np.exp(-(x-ideal_mu)**2/(2*ideal_sigma**2))
    plt.figure(figsize=(8,5))
    plt.plot(x,r,c='k',lw=3)
    r= r+np.random.normal(loc=noise_mean,scale=noise_sd,size=N_samples)
    plt.scatter(x,r,edgecolors='k',c='yellow',s=60)
    plt.grid(True)
    plt.show()
    return (r)
y=interactive(func,amplitude=[1,2,3,4,5],ideal_mu=(-5,5,0.5),
              ideal_sigma=(0,2,0.2),
              noise_sd=(0,1,0.1),noise_mean=(-1,1,0.2))
display(y)
